## Day 16

https://adventofcode.com/2020/day/16

In [1]:
import collections

In [2]:
import aocd

In [3]:
class Rule:
    def __init__(self, text):
        name, value = text.split(': ')
        self.name = name
        left, right = value.split(' or ')
        left = [int(s) for s in left.split('-')]
        self.left = range(left[0], left[1] + 1)
        right = [int(s) for s in right.split('-')]
        self.right = range(right[0], right[1] + 1)
        
    def __contains__(self, value):
        return value in self.left or value in self.right
    
    def __repr__(self):
        return (
            f'<{self.__class__.__name__}('
            f'{self.name}, {self.left}, {self.right})>'
        )

In [4]:
class Rules(collections.abc.Mapping):
    def __init__(self, rules):
        self._rules = {
            rule.name: rule
            for rule in rules
        }
        
    def __contains__(self, value):
        return any(
            value in rule.left or value in rule.right
            for rule in self._rules.values()
        )
    
    def is_valid(self, ticket):
        return all(n in self for n in ticket)

    @property
    def names(self):
        return list(self._rules.keys())
    
    def __len__(self):
        return len(self._rules)
    
    def __iter__(self):
        return iter(self._rules.values())
    
    def __getitem__(self, key):
        return self._rules[key]
    
    def __repr__(self):
        return (
            f'<{self.__class__.__name__}('
            f'{list(self._rules.values())})>'
        )

In [5]:
class Ticket(collections.abc.Sequence):
    def __init__(self, text):
        self.numbers = [int(s) for s in text.split(',')]
    
    def __repr__(self):
        return f'<{self.__class__.__name__}({self.numbers})>'
    
    def __iter__(self):
        return iter(self.numbers)
    
    def __len__(self):
        return len(self.numbers)
    
    def __getitem__(self, index_or_slice):
        return self.numbers[index_or_slice]

In [6]:
def get_data():
    raw = aocd.get_data(day=16, year=2020).split('\n\n')
    rules = Rules([Rule(s) for s in raw[0].splitlines()])
    ticket = Ticket(raw[1].splitlines()[1])
    nearby = [Ticket(s) for s in raw[2].splitlines()[1:]]
    return rules, ticket, nearby

In [7]:
rules, ticket, nearby = get_data()
rules

<Rules([<Rule(departure location, range(35, 797), range(811, 954))>, <Rule(departure station, range(25, 225), range(248, 953))>, <Rule(departure platform, range(47, 868), range(885, 960))>, <Rule(departure track, range(44, 122), range(127, 950))>, <Rule(departure date, range(49, 155), range(180, 961))>, <Rule(departure time, range(35, 533), range(546, 972))>, <Rule(arrival location, range(41, 701), range(706, 954))>, <Rule(arrival station, range(25, 563), range(568, 969))>, <Rule(arrival platform, range(31, 673), range(680, 970))>, <Rule(arrival track, range(43, 837), range(852, 962))>, <Rule(class, range(38, 292), range(304, 969))>, <Rule(duration, range(31, 747), range(755, 957))>, <Rule(price, range(46, 712), range(719, 972))>, <Rule(route, range(35, 585), range(608, 956))>, <Rule(row, range(39, 619), range(640, 951))>, <Rule(seat, range(25, 309), range(334, 955))>, <Rule(train, range(26, 902), range(913, 958))>, <Rule(type, range(33, 131), range(142, 966))>, <Rule(wagon, range(34, 

In [8]:
ticket

<Ticket([97, 103, 89, 191, 73, 79, 83, 101, 151, 71, 149, 53, 181, 59, 61, 67, 113, 109, 107, 127])>

In [9]:
nearby[0]

<Ticket([895, 527, 676, 768, 695, 821, 473, 414, 835, 426, 741, 650, 886, 709, 938, 355, 113, 358, 106, 888])>

In [10]:
assert all(len(x) == len(ticket) for x in nearby)

### Solution to Part 1

In [11]:
def scanning_error_rate(ticket, *, rules) -> int:
    return sum(n for n in ticket if n not in rules)

In [12]:
sum(scanning_error_rate(ticket, rules=rules) for ticket in nearby)

23115

### Solution to Part 2

In [13]:
def find_valid_tickets(tickets, *, rules):
    return set(ticket for ticket in tickets if rules.is_valid(ticket))

In [14]:
assert len(find_valid_tickets(nearby, rules=rules)) == 190

In [15]:
def group_field_values(tickets):
    values = []
    tickets = iter(tickets)
    first = next(tickets)
    for n in first:
        values.append([n])
    for ticket in tickets:
        for field, value in enumerate(ticket):
            values[field].append(value)
    return values

In [16]:
def possible_fields(rule, *, field_values):
    return set(
        field for field, values in enumerate(field_values)
        if all(value in rule for value in values)
    )

In [17]:
def find_possible_rules(*, tickets, rules):
    valid_tickets = find_valid_tickets(tickets, rules=rules)
    field_values = group_field_values(valid_tickets)
    return {
        rule.name:  possible_fields(rule, field_values=field_values)
        for rule in rules
    }

In [18]:
def valid_rules(*, tickets, rules):
    identified = {}
    possible = find_possible_rules(tickets=tickets, rules=rules)
    names = sorted(possible, key=lambda x: len(possible[x]))
    for name in names:
        field = [x for x in possible[name] if x not in identified]
        assert len(field) == 1
        field = field[0]
        identified[field] = name
    return identified

In [19]:
def ticket_values(ticket, *, nearby, rules):
    fields = valid_rules(tickets=nearby, rules=rules)
    return {
        name: ticket[field]
        for field, name in fields.items()
    }

In [20]:
ticket_values(ticket, nearby=nearby, rules=rules)

{'zone': 127,
 'arrival location': 107,
 'price': 79,
 'wagon': 71,
 'row': 59,
 'train': 103,
 'route': 109,
 'arrival platform': 97,
 'departure platform': 89,
 'departure station': 61,
 'departure track': 73,
 'departure time': 113,
 'departure date': 101,
 'departure location': 53,
 'arrival station': 83,
 'duration': 191,
 'seat': 149,
 'type': 151,
 'arrival track': 67,
 'class': 181}

In [21]:
89 * 61 * 73 * 113 * 101 * 53

239727793813